In [2]:
import os
import numpy as np
from model import *
from load import load_mnist_with_valid_set
import time 
import scipy
import sys
from sklearn.decomposition import PCA
from skimage.feature import hog

n_epochs = 1000
learning_rate = 0.0002
batch_size = 128
image_shape = [28,28,1]
dim_z = 100
dim_W1 = 1024
dim_W2 = 128
dim_W3 = 64
dim_channel = 1

# requirements
# /vis folder
# /data folder
# /models folder
# pip install pillow
# pip install scikit-image
# pip install scikit-learn

# to run:
# source activate tensorflow_p36 && python train_small.py 0.01 && python MC_Attacks_MNIST.py 10000 500 1 10 && sudo shutdown -P now
# source activate tensorflow_p36 && python train_small.py 0.1 && python MC_Attacks_MNIST.py 10000 500 1 10 && sudo shutdown -P now

# LOCAL:
# python MC_Attacks_MNIST.py 1000 500 1 1
# AMI: 
# python MC_Attacks_MNIST.py 10000 500 1 5

mc_sample_size = 100 #usually 10.000 samples per batch
model_no = '500' # which model to attack
instance_no = 33
exp_nos = int(1) # how many different experiments ofr specific indexes

data_dir = 'data/'

experiment = 'MC_Attacks_MNIST'

# only give the DCGAN 10% of training data
train_inds = np.loadtxt('train_inds.csv').astype(int)
percentage = np.loadtxt('percentage.csv')
trX, vaX, teX, trY, vaY, teY = load_mnist_with_valid_set(train_inds, percentage=percentage, data_dir=data_dir)

dcgan_model = DCGAN(
        batch_size=batch_size,
        image_shape=image_shape,
        dim_z=dim_z,
        dim_W1=dim_W1,
        dim_W2=dim_W2,
        dim_W3=dim_W3,
        )

Z_tf, Y_tf, image_tf, d_cost_tf, g_cost_tf, p_real, p_gen = dcgan_model.build_model()
sess = tf.InteractiveSession()

saver = tf.train.Saver()

saver = tf.train.import_meta_graph('models/mnist_gan.ckpt-'+model_no+'.meta')

saver.restore(sess, './models/mnist_gan.ckpt-'+model_no)

dt = np.dtype([('instance_no', int),
               ('exp_no', int),
               ('method', int), # 1 = white box, 2 = euclidean_PCA, 3 = hog, 4 = euclidean_PCA category, 5 = hog category, 6 = ais
               ('pca_n', int),
               ('percentage_of_data', float),
               ('percentile', float),
               ('mc_euclidean_no_batches', int), # stuff
               ('mc_hog_no_batches', int), # stuff
               ('sigma_ais', float),
               ('11_perc_mc_attack_log', float),
               ('11_perc_mc_attack_eps', float),
               ('11_perc_mc_attack_frac', float), 
               ('50_perc_mc_attack_log', float), 
               ('50_perc_mc_attack_eps', float),
               ('50_perc_mc_attack_frac', float),
               ('50_perc_white_box', float),
               ('11_perc_white_box', float),
               ('50_perc_ais', float),
               ('50_perc_ais_acc_rate', float),
              ])

experiment_results = []

def OneHot(X, n=10, negative_class=0.):
    X = np.asarray(X).flatten()
    Xoh = np.ones((len(X), n)) * negative_class
    Xoh[np.arange(len(X)), X] = 1.
    return Xoh

# random numbers
def generate_samples(sample_size=100):
    Z_np_sample = np.random.randn(sample_size, dim_z)
    Y_np_sample = OneHot( np.random.randint(10, size=[sample_size]))

    Z_tf_sample, Y_tf_sample, image_tf_sample = dcgan_model.samples_generator(batch_size=sample_size)

    generated_samples = sess.run(
                    image_tf_sample,
                    feed_dict={
                        Z_tf_sample:Z_np_sample,
                        Y_tf_sample:Y_np_sample
                        })
    generated_samples = (generated_samples + 1.)/2.
    return generated_samples

# indexes 1,11,21,31,... are ones, 2,12,22 are twos etc.
def generate_samples_for_digits(sample_size=100):
    
    Z_np_sample = np.random.randn(sample_size, dim_z)
    
    digits = np.zeros((sample_size,)).astype(int)
    for i in range(len(digits)):
        digits[i] = i%10
    Y_np_sample = OneHot( digits)

    Z_tf_sample, Y_tf_sample, image_tf_sample = dcgan_model.samples_generator(batch_size=sample_size)

    generated_samples = sess.run(
                    image_tf_sample,
                    feed_dict={
                        Z_tf_sample:Z_np_sample,
                        Y_tf_sample:Y_np_sample
                        })
    generated_samples = (generated_samples + 1.)/2.
    return generated_samples

def print_elapsed_time():
    end_time = int(time.time())
    d = divmod(end_time-start_time,86400)  # days
    h = divmod(d[1],3600)  # hours
    m = divmod(h[1],60)  # minutes
    s = m[1]  # seconds

    print('Elapsed Time: %d days, %d hours, %d minutes, %d seconds' % (d[0],h[0],m[0],s))

def discriminate_for_wb(data_to_be_discriminated, labels_to_be_discriminated, training_indicator):
    disc_results = np.zeros((len(data_to_be_discriminated),2))
    
    data_to_be_discriminated = data_to_be_discriminated.reshape( [-1, 28, 28, 1]) / 255
    
    disc_results[:,1] = training_indicator
    
    for iteration in range(len(data_to_be_discriminated) // batch_size):
        start = iteration*batch_size
        end = (iteration+1)*batch_size
        
        ind = np.arange(start,end)
        Xs = tf.to_float(data_to_be_discriminated[ind].reshape( [-1, 28, 28, 1]) / 255)
        Ys = tf.to_float(OneHot(labels_to_be_discriminated[ind]))
        disc_results[ind, 0] = np.reshape(sess.run(dcgan_model.discriminate(Xs ,Ys)),(batch_size,))
    
    # fill last few elements
    ind = np.arange(len(data_to_be_discriminated)-batch_size,len(data_to_be_discriminated))
    Xs = tf.to_float(data_to_be_discriminated[ind].reshape( [-1, 28, 28, 1]) / 255)
    Ys = tf.to_float(OneHot(labels_to_be_discriminated[ind]))
    disc_results[ind, 0] = np.reshape(sess.run(dcgan_model.discriminate(Xs ,Ys)),(batch_size,))
    
    return disc_results

def wb_attack_sample(disc_results_train, disc_results_validate):
    results = np.concatenate((disc_results_train,disc_results_validate))
    np.random.shuffle(results)
    results = results[results[:,0].argsort()]

    return results[-len(disc_results_train):,1].mean()

def wb_attack(trX_inds, vaX_inds, exp_no):

    disc_results_train = discriminate_for_wb(trX[trX_inds],trY[trX_inds],1)
    disc_results_validate = discriminate_for_wb(vaX[vaX_inds],vaY[vaX_inds],0)

    fifty_perc_wb_attack = wb_attack_sample(disc_results_train, disc_results_validate)

    iterations = 1000
    results_attacks = np.zeros((iterations, ))

    for i in range(len(results_attacks)):
        np.random.shuffle(disc_results_train)
        results_attacks[i] = wb_attack_sample(disc_results_train[0:10], disc_results_validate)

    eleven_perc_wb_attack = results_attacks.mean()

    print('50_perc_wb_attack: %.3f'%(fifty_perc_wb_attack))
    print('11_perc_wb_attack: %.3f'%(eleven_perc_wb_attack))

    # white box
    new_row = np.zeros(1, dtype = dt)[0]
    new_row['instance_no'] = instance_no
    new_row['exp_no'] = exp_no
    new_row['method'] = 1 # white box
    new_row['percentage_of_data'] = percentage
    new_row['50_perc_white_box'] = fifty_perc_wb_attack
    new_row['11_perc_white_box'] = eleven_perc_wb_attack
    experiment_results.append(new_row)
    np.savetxt(experiment+'.csv', np.array(experiment_results, dtype = dt))

def calculate_results_matrices(distances_real_vs_sample,distances_real_vs_train, d_min=0.1):

    results_sample = np.zeros((len(distances_real_vs_sample),4))
    for i in range(len(results_sample)):
        # indicate that dataset is a sample
        results_sample[i][0] = 0
        
        integral_approx = 0
        integral_approx_log = 0
        integral_approx_eps = 0
        for eps in distances_real_vs_sample[i]:
            if eps < d_min:
                integral_approx = integral_approx + d_min/eps
                integral_approx_log = integral_approx_log + (-np.log(eps/d_min))
                integral_approx_eps = integral_approx_eps + 1

        integral_approx = integral_approx/len(distances_real_vs_sample[0])
        integral_approx_log = integral_approx_log/len(distances_real_vs_sample[0])
        integral_approx_eps = integral_approx_eps/len(distances_real_vs_sample[0])

        results_sample[i][1] = integral_approx_log
        results_sample[i][2] = integral_approx_eps
        results_sample[i][3] = integral_approx

    results_train = np.zeros((len(distances_real_vs_train),4))
    for i in range(len(results_train)):
        # indicate that dataset is a training data set
        results_train[i][0] = 1
        
        integral_approx = 0
        integral_approx_log = 0
        integral_approx_eps = 0
        for eps in distances_real_vs_train[i]:
            if eps < d_min:
                integral_approx = integral_approx + d_min/eps
                integral_approx_log = integral_approx_log + (-np.log(eps/d_min))
                integral_approx_eps = integral_approx_eps + 1

        integral_approx = integral_approx/len(distances_real_vs_train[0])
        integral_approx_log = integral_approx_log/len(distances_real_vs_train[0])
        integral_approx_eps = integral_approx_eps/len(distances_real_vs_train[0])

        results_train[i][1] = integral_approx_log
        results_train[i][2] = integral_approx_eps
        results_train[i][3] = integral_approx
        
    return results_sample,results_train

def mc_attack_sample(results_sample, results_train):
    results = np.concatenate((results_sample, results_train))
    np.random.shuffle(results)
    mc_attack_log = results[results[:,1].argsort()][:,0][-len(results_train):].mean()
    np.random.shuffle(results)
    mc_attack_eps = results[results[:,2].argsort()][:,0][-len(results_train):].mean()
    np.random.shuffle(results)
    mc_attack_frac = results[results[:,3].argsort()][:,0][-len(results_train):].mean()

    return mc_attack_log, mc_attack_eps, mc_attack_frac

def mc_attack(results_sample, results_train):

    mc_attack_log, mc_attack_eps, mc_attack_frac = mc_attack_sample(results_sample, results_train)

    print('50_perc_mc_attack_log: %.3f'%(mc_attack_log))
    print('50_perc_mc_attack_eps: %.3f'%(mc_attack_eps))
    print('50_perc_mc_attack_frac: %.3f'%(mc_attack_frac))

    iterations = 1000
    results_attacks = np.zeros((iterations, 3))

    for i in range(len(results_attacks)):
        np.random.shuffle(results_train)
        results_attacks[i] = mc_attack_sample(results_sample, results_train[0:10])

    print('11_perc_mc_attack_log: %.3f'%(results_attacks[:,0].mean()))
    print('11_perc_mc_attack_eps: %.3f'%(results_attacks[:,1].mean()))
    print('11_perc_mc_attack_frac: %.3f'%(results_attacks[:,2].mean()))

    return mc_attack_log, mc_attack_eps, mc_attack_frac, results_attacks[:,0].mean(), results_attacks[:,1].mean(), results_attacks[:,2].mean()

def euclidean_PCA_mc_attack_category(n_components_pca, trX_inds, vaX_inds, exp_no, mc_euclidean_no_batches, percentiles):
    pca = PCA(n_components=n_components_pca)

    pca.fit_transform(teX.reshape((len(teX),784)))

    euclidean_trX = np.reshape(trX, (len(trX),784,))
    euclidean_trX = euclidean_trX[trX_inds]
    euclidean_trX = pca.transform(euclidean_trX)

    euclidean_vaX = np.reshape(vaX, (len(vaX),784,))
    euclidean_vaX = euclidean_vaX[vaX_inds]
    euclidean_vaX = pca.transform(euclidean_vaX)

    distances_trX = np.zeros((len(euclidean_trX), mc_euclidean_no_batches*mc_sample_size))
    distances_vaX = np.zeros((len(euclidean_vaX), mc_euclidean_no_batches*mc_sample_size))

    for i in range(mc_euclidean_no_batches):

        print('Working on %d/%d'%(i, mc_euclidean_no_batches))

        euclidean_generated_samples = generate_samples(mc_sample_size)

        euclidean_generated_samples = euclidean_generated_samples - euclidean_generated_samples.min()
        euclidean_generated_samples = euclidean_generated_samples*255/euclidean_generated_samples.max()
        euclidean_generated_samples = np.reshape(euclidean_generated_samples, (len(euclidean_generated_samples),784,))
        euclidean_generated_samples = pca.transform(euclidean_generated_samples)
        
        distances_trX_partial = np.zeros((len(euclidean_trX), mc_sample_size/10))
        distances_vaX_partial = np.zeros((len(euclidean_vaX), mc_sample_size/10))
        
        for j in range(len(distances_trX_partial)):
            # digit of current training example
            digit = trY[trX_inds[j]]
            # only compare to current digit
            distances_trX_partial[j,:] = scipy.spatial.distance.cdist(euclidean_trX[j], euclidean_generated_samples[[digit+10*i for i in range(mc_sample_size//10)]], 'euclidean')
        
        for j in range(len(distances_vaX_partial)):
            # digit of current training example
            digit = vaY[vaX_inds[j]]
            # only compare to current digit
            distances_vaX_partial[j,:] = scipy.spatial.distance.cdist(euclidean_vaX[j], euclidean_generated_samples[[digit+10*i for i in range(mc_sample_size//10)]], 'euclidean')
        
        #distances_trX_partial = scipy.spatial.distance.cdist(euclidean_trX, euclidean_generated_samples, 'euclidean')
        #distances_vaX_partial = scipy.spatial.distance.cdist(euclidean_vaX, euclidean_generated_samples, 'euclidean')

        # optimized, better than concatenate
        distances_trX[:,i*mc_sample_size:(i+1)*mc_sample_size] = distances_trX_partial
        distances_vaX[:,i*mc_sample_size:(i+1)*mc_sample_size] = distances_vaX_partial
        
        print_elapsed_time()

    for percentile in percentiles:
        print_elapsed_time()
        print('Calculating Results Matrices for '+str(percentile)+' Percentile...')

        d_min = np.percentile(np.concatenate((distances_trX,distances_vaX)),percentile)
        results_sample,results_train = calculate_results_matrices(distances_vaX, distances_trX,d_min)
        
        # save data
        new_row = np.zeros(1, dtype = dt)[0]
        new_row['instance_no'] = instance_no
        new_row['exp_no'] = exp_no
        new_row['method'] = 4 # euclidean PCA cat
        new_row['pca_n'] = n_components_pca
        new_row['percentage_of_data'] = percentage
        new_row['percentile'] = percentile
        new_row['mc_euclidean_no_batches'] = mc_euclidean_no_batches

        mc_attack_results = mc_attack(results_sample, results_train)
        new_row['50_perc_mc_attack_log'] = mc_attack_results[0]
        new_row['50_perc_mc_attack_eps'] = mc_attack_results[1]
        new_row['50_perc_mc_attack_frac'] = mc_attack_results[2]
        new_row['11_perc_mc_attack_log'] = mc_attack_results[3]
        new_row['11_perc_mc_attack_eps'] = mc_attack_results[4]
        new_row['11_perc_mc_attack_frac'] = mc_attack_results[5]
        
        experiment_results.append(new_row)
        np.savetxt(experiment+'.csv', np.array(experiment_results, dtype = dt))

def generate_batch_hog_features(samples):
    features_matrix = np.zeros((len(samples),81))

    for i in range(len(samples)):
        features_matrix[i] = hog(samples[i].reshape((28, 28)), orientations=9, pixels_per_cell=(9, 9), visualise=False) #, transform_sqrt=True, block_norm='L2-Hys')
    
    return features_matrix

def hog_mc_attack(trX_inds, vaX_inds, exp_no, mc_hog_no_batches, percentiles):

    feature_matrix_vaX = generate_batch_hog_features(vaX[vaX_inds])
    feature_matrix_trX = generate_batch_hog_features(trX[trX_inds])

    distances_trX = np.zeros((len(feature_matrix_trX), mc_hog_no_batches*mc_sample_size))
    distances_vaX = np.zeros((len(feature_matrix_vaX), mc_hog_no_batches*mc_sample_size))

    for i in range(mc_hog_no_batches):

        print('Working on %d/%d'%(i, mc_hog_no_batches))

        generated_samples = generate_samples(mc_sample_size)

        generated_samples = generated_samples - generated_samples.min()
        generated_samples = generated_samples*255/generated_samples.max()

        feature_matrix_generated = generate_batch_hog_features(generated_samples)

        distances_trX_partial = scipy.spatial.distance.cdist(feature_matrix_trX, feature_matrix_generated, 'euclidean')
        distances_vaX_partial = scipy.spatial.distance.cdist(feature_matrix_vaX, feature_matrix_generated, 'euclidean')

        # optimized, better than concatenate
        distances_trX[:,i*mc_sample_size:(i+1)*mc_sample_size] = distances_trX_partial
        distances_vaX[:,i*mc_sample_size:(i+1)*mc_sample_size] = distances_vaX_partial

        print_elapsed_time()

    for percentile in percentiles:
        print_elapsed_time()
        print('Calculating Results Matrices for '+str(percentile)+' Percentile...')

        d_min = np.percentile(np.concatenate((distances_trX,distances_vaX)),percentile)
        results_sample,results_train = calculate_results_matrices(distances_vaX, distances_trX,d_min)

        # save data
        new_row = np.zeros(1, dtype = dt)[0]
        new_row['instance_no'] = instance_no
        new_row['exp_no'] = exp_no
        new_row['method'] = 3
        new_row['percentage_of_data'] = percentage
        new_row['percentile'] = percentile
        new_row['mc_hog_no_batches'] = mc_hog_no_batches

        mc_attack_results = mc_attack(results_sample, results_train)
        new_row['50_perc_mc_attack_log'] = mc_attack_results[0]
        new_row['50_perc_mc_attack_eps'] = mc_attack_results[1]
        new_row['50_perc_mc_attack_frac'] = mc_attack_results[2]
        new_row['11_perc_mc_attack_log'] = mc_attack_results[3]
        new_row['11_perc_mc_attack_eps'] = mc_attack_results[4]
        new_row['11_perc_mc_attack_frac'] = mc_attack_results[5]
        
        experiment_results.append(new_row)
        np.savetxt(experiment+'.csv', np.array(experiment_results, dtype = dt))

start_time = int(time.time())

ValueError: At least two variables have the same name: discrim_W1

In [5]:
mc_hog_no_batches = 100
mc_euclidean_no_batches = 100

for exp_no in range(exp_nos):

    trX_inds = np.arange(len(trX))
    np.random.shuffle(trX_inds)
    trX_inds = trX_inds[0:100]

    vaX_inds = np.arange(len(trX))
    np.random.shuffle(vaX_inds)
    vaX_inds = vaX_inds[0:100]

    # euclidean pca mc attack
    euclidean_PCA_mc_attack_category(42, trX_inds, vaX_inds, exp_no, mc_euclidean_no_batches, [1,0.1,0.01,0.001,0.0001])
    print(experiment+': Finished PCA Monte Carlo in experiment %d of %d'%(exp_no+1, exp_nos))

    print_elapsed_time()
    
print(experiment_results)

Working on 0/100


FailedPreconditionError: Attempting to use uninitialized value gen_W4_1
	 [[Node: gen_W4_1/read = Identity[T=DT_FLOAT, _class=["loc:@gen_W4_1"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](gen_W4_1)]]

Caused by op 'gen_W4_1/read', defined at:
  File "c:\users\d065042\appdata\local\conda\conda\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\d065042\appdata\local\conda\conda\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\d065042\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\d065042\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\d065042\appdata\local\conda\conda\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.s